1. Import libraries



In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

2. Prepare data

In [12]:
df = pd.read_csv('Nuova_Simonelli.csv')
display(df)

,customer_id,cafe_access_index,internet_speed_index,months_since_last_supply_order,supply_orders,total_spent,months_since_last_machine_purchase,old_generation_model,early_adopter,multiboiler_system,cafe_type,income,housing_value_zip,buy1,training
0,34009,80.740450,8.304583,13,2,222,5,0,0,0,chain_owned_cafe,0.425687,1.116758,0,1
1,46579,6.423846,2.829486,24,2,135,12,0,0,0,chain_owned_cafe,0.462444,2.240519,1,1
2,80418,85.343426,7.914067,23,1,36,2,0,0,0,independent_cafe,0.437185,0.877786,0,1
3,83436,74.427489,37.928191,14,1,14,3,0,0,0,chain_owned_cafe,0.648146,2.226043,0,1
4,95388,65.636565,1.939508,13,1,38,22,0,0,1,chain_owned_cafe,0.568095,1.240980,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,48969,40.491580,2.796824,30,5,102,20,0,0,0,chain_owned_cafe,0.312351,1.205787,0,0
44996,49497,44.695969,2.592837,26,4,183,7,0,0,0,chain_owned_cafe,0.648587,1.076492,0,0
44997,68910,7.477873,0.594275,14,2,222,27,0,1,0,other,0.394039,1.399797,0,0
44998,79655,95.297577,10.435415,14,4,175,27,0,1,0,chain_owned_cafe,0.550879,3.461171,0,0


In [13]:
#1 Drop one category to avoid redundancy (and perfect collinearity) and create dummy variables for cafe_type
df_encoded = pd.get_dummies(df, columns=['cafe_type'], drop_first=True)


In [14]:
# 2. Clean data
# ---------------------------------------------------------
dependent_variable = 'buy1'

# Apply dropna to df_encoded, which is the dataframe we intend to use for modeling
df_encoded.dropna(subset=[dependent_variable], inplace=True)

# 3. Train/split dataset split
# ---------------------------------------------------------
# training = 1 → training sample
# training = 0 → validation sample
train = df_encoded[df_encoded['training'] == 1].copy()
valid = df_encoded[df_encoded['training'] == 0].copy()

# ---------------------------------------------------------
# 4. DEFINE X AND Y FOR TRAIN / VALIDATION
# ---------------------------------------------------------
# These lines are redundant as train/valid are already defined above.
# train = df[df['training'] == 1].copy()
# valid = df[df['training'] == 0].copy()

y_train = train[dependent_variable]
y_valid = valid[dependent_variable]

# Define independent variables based on df_encoded columns
drop_cols = ['customer_id', dependent_variable, 'training']
independent_variables = [col for col in df_encoded.columns if col not in drop_cols]

X_train = train[independent_variables]
X_valid = valid[independent_variables]

# Add intercept (constant) term for logistic regression
X_train_const = sm.add_constant(X_train).astype(float)
X_valid_const = sm.add_constant(X_valid).astype(float)

In [15]:
# 5. FIT LOGISTIC REGRESSION (ON TRAINING ONLY)
# ---------------------------------------------------------
logit_model = sm.Logit(y_train, X_train_const)
logit_result = logit_model.fit()

# See detailed coefficient table (for interpretation in write-up)
print(logit_result.summary())


Optimization terminated successfully.
         Current function value: 0.147555
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                   buy1   No. Observations:                31500
Model:                          Logit   Df Residuals:                    31486
Method:                           MLE   Df Model:                           13
Date:                Thu, 13 Nov 2025   Pseudo R-squ.:                  0.1404
Time:                        07:34:10   Log-Likelihood:                -4648.0
converged:                       True   LL-Null:                       -5407.3
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -5.4954      0.150    

In [16]:
# 7. PREDICT PROBABILITIES ON VALIDATION SAMPLE (ROUND 1)
# ---------------------------------------------------------
valid_pred_round1 = logit_result.predict(X_valid_const)

valid

,customer_id,cafe_access_index,internet_speed_index,months_since_last_supply_order,supply_orders,total_spent,months_since_last_machine_purchase,old_generation_model,early_adopter,multiboiler_system,income,housing_value_zip,buy1,training,cafe_type_independent_cafe,cafe_type_other
31500,62932,0.000000,0.000000,6,2,102,27,1,0,1,0.380212,1.592624,0,0,False,False
31501,18602,41.901487,3.319568,12,1,25,36,0,1,0,0.371186,3.605585,0,0,True,False
31502,44844,3.528961,0.410157,19,3,212,8,0,0,1,0.271371,0.292552,0,0,False,False
31503,69051,90.479468,8.973667,11,5,512,17,0,0,1,0.499208,1.657061,0,0,False,True
31504,90364,50.374928,3.040005,40,4,152,19,0,0,0,0.632790,5.218886,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,48969,40.491580,2.796824,30,5,102,20,0,0,0,0.312351,1.205787,0,0,False,False
44996,49497,44.695969,2.592837,26,4,183,7,0,0,0,0.648587,1.076492,0,0,False,False
44997,68910,7.477873,0.594275,14,2,222,27,0,1,0,0.394039,1.399797,0,0,False,True
44998,79655,95.297577,10.435415,14,4,175,27,0,1,0,0.550879,3.461171,0,0,False,False


  ECONOMIC LOGIC FOR ROUND 2
  

In [17]:
contact_cost = 64          # cost to contact a customer in Round 2
profit_per_upgrade = 2500  # profit from a successful upgrade (net of product costs)

# Break-even response rate: when expected revenue = contact cost
r_break_even = contact_cost / profit_per_upgrade  # 64 / 2500 = 0.0256
print("Break-even response rate (Round 2):", r_break_even)

Break-even response rate (Round 2): 0.0256


In [18]:
# Adjust probability for round 2
def adjust_to_round2(probs_round1):
    return probs_round1 / 2.0

# Evaluate round 2 profit
def evaluate_profit_round2(y_true_round1, probs_round1, threshold=r_break_even):
    probs_round2 = adjust_to_round2(probs_round1)
    target_mask = probs_round2 >= threshold
    if target_mask.sum() == 0:
        return 0, 0, 0
    # Approximate Round 2 response using Round 1 behavior
    # Among targeted customers, calculate actual Round 1 response rate
    round1_response = y_true_round1[target_mask].mean()
    # Adjust to Round 2 response (case assumption)
    round2_response = round1_response / 2.0

    # Expected number of successful upgrades in Round 2
    expected_upgrades = round2_response * target_mask.sum()

    # Revenue from these upgrades
    revenue = expected_upgrades * profit_per_upgrade

    # Cost of contacting all targeted customers
    total_cost = target_mask.sum() * contact_cost

    # Net profit
    profit = revenue - total_cost

    return profit, target_mask.sum(), round2_response

In [19]:
# COMPUTE EXPECTED ROUND 2 PROFIT FOR THIS MODEL
# ---------------------------------------------------------
profit, n_targeted, approx_r2 = evaluate_profit_round2(
    y_true_round1=y_valid.values,
    probs_round1=valid_pred_round1,
    threshold=r_break_even
)

print("\n=== Economic Performance (Validation Sample, Round 2) ===")
print(f"Total expected profit: ${profit:,.0f}")
print(f"Number of customers targeted: {n_targeted}")
print(f"Approx. Round 2 response rate among targeted: {approx_r2:.4f}")


=== Economic Performance (Validation Sample, Round 2) ===
Total expected profit: $251,864
Number of customers targeted: 3174
Approx. Round 2 response rate among targeted: 0.0573


RANDOM FOREST

In [20]:
from sklearn.ensemble import RandomForestClassifier

# ---------------------------------------------------------
# RANDOM FOREST MODEL (TRAINING ON TRAIN SAMPLE ONLY)
# ---------------------------------------------------------

# Features for train and validation (same independent_variables as before)
X_train_rf = train[independent_variables]
X_valid_rf = valid[independent_variables]

# Define the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=300,   # number of trees
    max_depth=8,       # limit depth to control overfitting
    random_state=42,
    n_jobs=-1          # use all cores (faster)
)

# Fit the model on training data
rf_model.fit(X_train_rf, y_train)

# Predict Round 1 probabilities on validation sample
# [:, 1] = probability of class 1 (buy1 = 1)
valid_pred_round1_rf = rf_model.predict_proba(X_valid_rf)[:, 1]


In [21]:
# ---------------------------------------------------------
# ECONOMIC EVALUATION FOR RANDOM FOREST (ROUND 2)
# ---------------------------------------------------------

rf_profit, rf_n_targeted, rf_r2 = evaluate_profit_round2(
    y_true_round1=y_valid.values,
    probs_round1=valid_pred_round1_rf,
    threshold=r_break_even
)

print("\n=== Random Forest: Economic Performance (Validation, Round 2) ===")
print(f"Total expected profit: ${rf_profit:,.0f}")
print(f"Number of customers targeted: {rf_n_targeted}")
print(f"Approx. Round 2 response rate among targeted: {rf_r2:.4f}")



=== Random Forest: Economic Performance (Validation, Round 2) ===
Total expected profit: $271,050
Number of customers targeted: 3675
Approx. Round 2 response rate among targeted: 0.0551


In [22]:
# ---------------------------------------------------------
# BUILD VALIDATION RESULTS DF FOR RANDOM FOREST
# ---------------------------------------------------------

rf_valid_results = valid[['customer_id', 'buy1']].copy()

# Round 1 and Round 2 predicted probabilities from Random Forest
rf_valid_results['pred_round1_rf'] = valid_pred_round1_rf
rf_valid_results['pred_round2_rf'] = rf_valid_results['pred_round1_rf'] / 2.0

# Targeting decision under RF model
rf_valid_results['target_round2_rf'] = (rf_valid_results['pred_round2_rf'] >= r_break_even).astype(int)

# Inspect top customers by predicted Round 2 probability
rf_valid_results.sort_values('pred_round2_rf', ascending=False).head(10)


,customer_id,buy1,pred_round1_rf,pred_round2_rf,target_round2_rf
38932,29313,0,0.499923,0.249962,1
34910,29640,1,0.446031,0.223015,1
43337,26364,1,0.441076,0.220538,1
35376,64734,1,0.438875,0.219438,1
44835,47547,0,0.437869,0.218934,1
35559,43701,1,0.421463,0.210732,1
38506,38854,1,0.418637,0.209319,1
44675,97448,1,0.409472,0.204736,1
38008,77764,0,0.405840,0.202920,1
35383,46798,0,0.401591,0.200795,1


NEURO NETWORK

In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# ---------------------------------------------------------
# NEURAL NETWORK MODEL (MLPClassifier)
# ---------------------------------------------------------

# 1) Features for train and validation (same as before)
X_train_nn = train[independent_variables]
X_valid_nn = valid[independent_variables]

# 2) Scale features (neural nets train better on scaled data)
scaler_nn = StandardScaler()
X_train_nn_scaled = scaler_nn.fit_transform(X_train_nn)
X_valid_nn_scaled = scaler_nn.transform(X_valid_nn)

# 3) Define the neural network
#    - hidden_layer_sizes=(16, 8): two hidden layers with 16 and 8 neurons
#    - activation='relu': common activation function
#    - max_iter=300: number of training iterations
#    - random_state=42: reproducibility
mlp_model = MLPClassifier(
    hidden_layer_sizes=(16, 8),
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

# 4) Fit the model on the training data
mlp_model.fit(X_train_nn_scaled, y_train)

# 5) Predict Round 1 probabilities on validation sample
valid_pred_round1_nn = mlp_model.predict_proba(X_valid_nn_scaled)[:, 1]


In [24]:
# ---------------------------------------------------------
# ECONOMIC EVALUATION FOR NEURAL NETWORK (ROUND 2)
# ---------------------------------------------------------

nn_profit, nn_n_targeted, nn_r2 = evaluate_profit_round2(
    y_true_round1=y_valid.values,
    probs_round1=valid_pred_round1_nn,
    threshold=r_break_even
)

print("\n=== Neural Network (MLP): Economic Performance (Validation, Round 2) ===")
print(f"Total expected profit: ${nn_profit:,.0f}")
print(f"Number of customers targeted: {nn_n_targeted}")
print(f"Approx. Round 2 response rate among targeted: {nn_r2:.4f}")



=== Neural Network (MLP): Economic Performance (Validation, Round 2) ===
Total expected profit: $266,602
Number of customers targeted: 3432
Approx. Round 2 response rate among targeted: 0.0567


GRADIENT BOOSTING

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

# ---------------------------------------------------------
# GRADIENT BOOSTING MODEL (TRAINING ON TRAIN SAMPLE ONLY)
# ---------------------------------------------------------

# Features for train and validation (same independent_variables as before)
X_train_gb = train[independent_variables]
X_valid_gb = valid[independent_variables]

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier(
    n_estimators=200,    # number of boosting stages (trees)
    learning_rate=0.1,   # step size shrinkage
    max_depth=3,         # depth of individual trees
    random_state=42
)

# Fit the model on training data
gb_model.fit(X_train_gb, y_train)

# Predict Round 1 probabilities on validation sample
# [:, 1] = probability of class 1 (buy1 = 1)
valid_pred_round1_gb = gb_model.predict_proba(X_valid_gb)[:, 1]


In [26]:
# ---------------------------------------------------------
# ECONOMIC EVALUATION FOR GRADIENT BOOSTING (ROUND 2)
# ---------------------------------------------------------

gb_profit, gb_n_targeted, gb_r2 = evaluate_profit_round2(
    y_true_round1=y_valid.values,
    probs_round1=valid_pred_round1_gb,
    threshold=r_break_even
)

print("\n=== Gradient Boosting: Economic Performance (Validation, Round 2) ===")
print(f"Total expected profit: ${gb_profit:,.0f}")
print(f"Number of customers targeted: {gb_n_targeted}")
print(f"Approx. Round 2 response rate among targeted: {gb_r2:.4f}")



=== Gradient Boosting: Economic Performance (Validation, Round 2) ===
Total expected profit: $261,754
Number of customers targeted: 3039
Approx. Round 2 response rate among targeted: 0.0601


DECISION TREES

In [27]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

# ---------------------------------------------------------
# Decision Tree MODEL (TRAINING ON TRAIN SAMPLE ONLY)
# ---------------------------------------------------------

# Features for train and validation (same independent_variables as before)
X_train_dt = train[independent_variables]
X_valid_dt = valid[independent_variables]

# Define the Decision Tree model
dt_model = DecisionTreeClassifier(
    max_depth=5,
    random_state=45
)

# Fit the model on training data
dt_model.fit(X_train_dt, y_train)

# Predict Round 1 probabilities on validation sample
# [:, 1] = probability of class 1 (buy1 = 1)
valid_pred_round1_dt = dt_model.predict_proba(X_valid_dt)[:, 1]

In [28]:
# ---------------------------------------------------------
# ECONOMIC EVALUATION FOR DECISION TREE (ROUND 2)
# ---------------------------------------------------------

dt_profit, dt_n_targeted, dt_r2 = evaluate_profit_round2(
    y_true_round1=y_valid.values,
    probs_round1=valid_pred_round1_dt,
    threshold=r_break_even
)

print("\n=== Decision Tree: Economic Performance (Validation, Round 2) ===")
print(f"Total expected profit: ${dt_profit:,.0f}")
print(f"Number of customers targeted: {dt_n_targeted}")
print(f"Approx. Round 2 response rate among targeted: {dt_r2:.4f}")


=== Gradient Boosting: Economic Performance (Validation, Round 2) ===
Total expected profit: $231,954
Number of customers targeted: 3114
Approx. Round 2 response rate among targeted: 0.0554


In [29]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# ---------------------------------------------------------
# BUILD MODEL COMPARISON TABLE
# ---------------------------------------------------------

comparison_rows = []

# 1) Logistic Regression
comparison_rows.append({
    'Model': 'Logistic Regression',
    'Profit_Round2': profit,
    'N_Targeted_Round2': n_targeted,
    'RespRate_Round2_approx': approx_r2,
    'AUC_Round1': roc_auc_score(y_valid, valid_pred_round1)
})

# 2) Random Forest
comparison_rows.append({
    'Model': 'Random Forest',
    'Profit_Round2': rf_profit,
    'N_Targeted_Round2': rf_n_targeted,
    'RespRate_Round2_approx': rf_r2,
    'AUC_Round1': roc_auc_score(y_valid, valid_pred_round1_rf)
})

# 3) Neural Network (MLP) - comment out if you didn't run it
comparison_rows.append({
    'Model': 'Neural Net (MLP)',
    'Profit_Round2': nn_profit,
    'N_Targeted_Round2': nn_n_targeted,
    'RespRate_Round2_approx': nn_r2,
    'AUC_Round1': roc_auc_score(y_valid, valid_pred_round1_nn)
})

# 4) Gradient Boosting
comparison_rows.append({
    'Model': 'Gradient Boosting',
    'Profit_Round2': gb_profit,
    'N_Targeted_Round2': gb_n_targeted,
    'RespRate_Round2_approx': gb_r2,
    'AUC_Round1': roc_auc_score(y_valid, valid_pred_round1_gb)
})

# 5) Decision Tree
comparison_rows.append({
    'Model': 'Decision Tree',
    'Profit_Round2': dt_profit,
    'N_Targeted_Round2': dt_n_targeted,
    'RespRate_Round2_approx': dt_r2,
    'AUC_Round1': roc_auc_score(y_valid, valid_pred_round1_dt)
})

# Turn into DataFrame
model_comparison_df = pd.DataFrame(comparison_rows)

# Sort by profit (highest first)
model_comparison_df = model_comparison_df.sort_values('Profit_Round2', ascending=False)

model_comparison_df


,Model,Profit_Round2,N_Targeted_Round2,RespRate_Round2_approx,AUC_Round1
1,Random Forest,271050.0,3675,0.055102,0.817255
2,Neural Net (MLP),266602.0,3432,0.056672,0.802446
3,Gradient Boosting,261754.0,3039,0.060053,0.815207
0,Logistic Regression,251864.0,3174,0.057341,0.797571
4,Decision Tree,231954.0,3114,0.055395,0.776266


All models deliver a predicted Round-2 response rate well above the 2.56% break-even threshold, roughly double or more. The more flexible models (Random Forest, Gradient Boosting, and MLP) also improve AUC relative to logistic regression, indicating better ranking of likely upgraders. In terms of the objective of maximizing Round-2 profit, the **Random Forest model performs best, yielding the highest expected profit ($271k)** while targeting 3,675 customers.

**Final model choice**

Based on this comparison, I would use the **Random Forest model to score customers for the Round-2 campaign**. It offers the highest expected profit on the out-of-sample validation data and a strong AUC, suggesting it captures useful non-linear patterns beyond the logistic regression baseline. I would **still use the logistic regression output for interpretability**—i.e., to communicate which variables are most strongly associated with upgrade propensity—but rely on the Random Forest scores to decide whom to target.

**Limitation**

1. Oversimplified assumption: Round 2 assume that probability is  half of Round 1, this is a major simplification. Customers declined the first offer for different reasons such as price or skepticism. A single uniform 50% reduction fails to capture this. This limits the prediction's accuracy.

2. Customer Annoyance: Re-contacting people who previously said "no" risks damaging customer relationships and brand perception. In this case, the analysis does not account for this potential negative impact.

3. Over-reliance on single metric: the economic analysis focused on short-term profit from the upgrade, incorporating LTV might provide a more accurate results of the upgrade's economic impact and lead to better targeting decisions.